실습: 사용자가 입력한 질문에 맞는 캠핑장을 검색해서 추천하는 ChatGPT 어플리케이션을 작성하시오(힌트: 고캠핑 API) 

In [3]:
import os
import openai
import requests
import json

openai.api_key = os.getenv("OPENAI_API_KEY")

# (가상) 고캠핑 API 키 및 엔드포인트
GO_CAMPING_API_KEY = "key"
GO_CAMPING_API_ENDPOINT = "http://apis.data.go.kr/B551011/GoCamping/basedList"  # 가상 URL

def recommend_campsites(query):
    """
    사용자 질문에 맞는 캠핑장을 검색하여 추천합니다.

    Args:
        query (str): 사용자 질문

    Returns:
        str: 추천 캠핑장 정보 문자열
    """

    try:
        # API 요청 파라미터 설정 (예시)
        params = {
            "serviceKey": GO_CAMPING_API_KEY,
            "numOfRows": 10,
            "pageNo": 1,
            "MobileOS": "ETC",
            "MobileApp": "AppTest",
            "_type": "json",  # 사용자 질문을 키워드로 사용
            # 필요한 다른 파라미터 추가 (지역, 편의시설 등)
        }

        # API 요청
        response = requests.get(GO_CAMPING_API_ENDPOINT, params=params)
        #print(response.url)
        response.raise_for_status()  # 오류 발생 시 예외 발생

        # 응답 데이터 파싱
        data = response.json()

        # OpenAI 프롬프트 생성
        camps = data['response']['body']['items']['item']
        camp_info_for_prompt = ""
        for camp in camps:
            if camp.get('lctCl') and '바다' in camp['lctCl'] or '해변' in camp['lctCl']:
                camp_info_for_prompt += f"캠핑장 이름: {camp['facltNm']}, 위치: {camp['doNm']} {camp['sigunguNm']}, 특징: {camp.get('lineIntro', '')}\n"
        
        prompt = f"""
        사용자는 "{query}"라고 질문했습니다. 다음 캠핑장 정보를 참고하여 바다 근처 캠핑장을 추천해주세요.

        {camp_info_for_prompt}
        """
        
        # OpenAI API 호출
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",  # 또는 다른 적절한 모델
            messages=[
                {"role": "system", "content": "캠핑장 추천 전문가"},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1000,
            temperature=0.7,  # 창의적인 답변을 위해 온도 조정
        )
        
        recommendations = response.choices[0].message.content
        return recommendations

    except requests.exceptions.RequestException as e:
        return f"API 요청 중 오류 발생: {e}"
    except Exception as e:
        return f"추천 중 오류 발생: {e}"


# 예시 사용
user_query = "계곡곡 근처 캠핑장 추천해줘"
recommendations = recommend_campsites(user_query)
print(f"사용자 질문: {user_query}\n{recommendations}")

사용자 질문: 바다 근처 캠핑장 추천해줘
바다 근처 캠핑장으로는 솔비치캠핑장을 추천해드립니다. 이 캠핑장은 충청남도 태안군에 위치해 있으며, 바다를 품고 자리하고 있어 해수욕과 바다 전망을 즐길 수 있는 캠핑장으로 유명합니다. 솔비치캠핑장은 자연환경이 아름다우며, 조용한 분위기에서 휴식을 취할 수 있도록 조성되어 있습니다. 바다 근처에서 자연 속에서 힐링하고 싶다면 솔비치캠핑장을 방문해보세요.
